In [1]:
import os
import json
import numpy as np
import torch 
import transformers
import warnings


from sklearn.model_selection import train_test_split
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

warnings.filterwarnings('ignore')

In [3]:
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [4]:
path = os.path.abspath('interim/3/train.json')

In [5]:
path = os.path.abspath('interim/3/train.json')

with open(path) as js:
    full_review = json.load(js)

In [6]:
review = [None] * len(full_review)

for i in range(len(full_review)):
    review[i] = full_review[i]['text']

## preprocessing

In [7]:
target = [None] * len(review)
main_review = [None] * len(review)

In [8]:
for i in range(len(review)):
    #delete general_impression from review
    main_review[i] = review[i]
    general_impression_idx = main_review[i].find('Общее впечатление')
    if general_impression_idx != -1:
        main_review[i] = main_review[i][:general_impression_idx]
        
    #delete everything before review
    review_idx = main_review[i].find('Отзыв: ')
    if review_idx != -1:
        main_review[i] = main_review[i][review_idx + len('Отзыв: '):]
        
    #target 
    target_idx = main_review[i].find('\n')
    target[i] = main_review[i][:target_idx]

    #delete "TEXT" and "\n" if they exist 
    TEXT_idx = main_review[i].rfind('\n')
    if TEXT_idx != -1:
        main_review[i] = main_review[i][TEXT_idx + len('\n'):]
    

In [9]:
train_review, test_review, train_target, test_target = train_test_split(main_review, target)

In [10]:
model_checkpoint = 'cointegrated/rut5-base-absum'

In [11]:
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
data = {'train':Dataset.from_dict({'summary':train_target,'text':train_review}),
     'test':Dataset.from_dict({'summary':test_target,'text':test_review})
     }

data = DatasetDict(data)

In [13]:
data

DatasetDict({
    train: Dataset({
        features: ['summary', 'text'],
        num_rows: 2200
    })
    test: Dataset({
        features: ['summary', 'text'],
        num_rows: 734
    })
})

In [29]:
max_intput_length = 1024
max_target_length = 128

In [30]:
def preprocess_function(examples):
    type(examples)
    model_inputs = tokenizer(examples["text"], max_length=max_intput_length, truncation=True, padding=True)

    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [31]:
tokenized_data = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/734 [00:00<?, ? examples/s]

In [32]:
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [106]:
training_args = transformers.Seq2SeqTrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=6,
    )

In [107]:
trainer = transformers.Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [108]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.964900,1.861135
2,2.070300,1.793932
3,1.906800,1.731060
4,1.820700,1.707372
5,1.783700,1.701762
6,1.694800,1.699989


TrainOutput(global_step=13200, training_loss=1.8768957704486269, metrics={'train_runtime': 2908.0423, 'train_samples_per_second': 4.539, 'train_steps_per_second': 4.539, 'total_flos': 9225833527296000.0, 'train_loss': 1.8768957704486269, 'epoch': 6.0})

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [109]:
text_example = data["test"]['text'][-1]
print(text_example)

Впервые использовала свечи Макмирор после аборта на сроке почти 3 месяца. Из меня еще продолжали выходить мелкие куски и начались признаки воспаления. Это общая слабость, плохой цвет лица, воспаленные прыщи по лицу и плечам. Проставила 8 дней свечи Макмирор и как рукой сняло. При чем ни каких антибиотиков параллельно не принимала. Очень рекомендую женщинам при несильных воспалителтных процессах женских половых органов.


In [110]:
 input_ids = tokenizer.encode(
    text_example,
    return_tensors="pt",
    max_length=1024,
    truncation=True,
    ).to(device)

In [111]:
summary_text_ids = model.generate(
    input_ids=input_ids,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    max_length=max_target_length,
    num_beams=4,
)

In [112]:
summary_text_ids

tensor([[    0, 14091,  2228, 19452, 29146, 10036,  5840,  2307,   313,  7911,
          1280, 20118,   311,   259,   264,   259, 14061,  1909,  5548,   411,
           805,   315, 15477,  8383,  4591,  1123, 11888,  3351,  2534,  3951,
          2681,   685,     1]], device='cuda:0')

In [113]:
decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
decoded_text

'Свечи ректальные Макмирор "150000 МЕ" - Хорошее средство при воспалительных процессах женских половых органов'

In [114]:
data['test']['summary'][-1]

'Суппозитории вагинальные Макмирор - Свечи Макмирор удобные в использовании не мнутся не растекаются в руках.'

In [115]:
import evaluate
from tqdm import tqdm

In [116]:
rouge = evaluate.load('rouge')

In [132]:
summaries = []

for text in tqdm(data["test"]['text']):
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        ).to(device)

    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        max_length=142,
        num_beams=4,
    )
    decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
    print(decoded_text)
    print("____________________")
    print()
    print(text)
    
    summaries.append(decoded_text)

  0%|                                           | 1/734 [00:00<03:45,  3.25it/s]

Таблетки Биотики "Бисакодил" - Хороший слабительный препарат
____________________
Давно страдаю запорами, не знаю причину. Обычно не обращала на это особого внимания, пока не появился геморрой, который стал следствием нарушения регулярности стула. Сделали операцию и предупредили, что стул должен быть регулярным иначе будут другие проблемы. За неимением желания тратить много денег на слабительные решила попробовать уладить свою проблему при помощи Бисакодила. Стоит он чуть больше 20 рублей, что несомненно порадовало, но возникло сомнение в его эффективности. Выпив одну таблетку на ночь, я не ощутила никакого эффекта вечером, но утром обалдела, он реально действенный. Однако днем следующего дня побаливал живот терпимо конечно, но неприятно.  После 2 таблетки у меня было ощущение будто меня выворачивает изнутри, боли были сильные, но и эффект тоже через чур. Решила, что нужно поменять данное средство на более мягкое в действии.


  0%|                                           | 2/734 [00:00<04:35,  2.66it/s]

Витамины для наружного применения "Глицин" - Не советую никому!
____________________
Должна сразу отметить, что с ужасным недоверием отношусь ко всем витаминам ( да и к лекарствам в целом) которые сделаны в России. Но мне их так рекомендовал врач! Говорила, что они одни из лучших и тем более маленькая цена. Тогда я еще не знала, что существуют просто потрясающие витамины, я уже писала о нем отзыв  . Сияние, нужно было пить 1 таблетку в день в течении месяца. Но уже спустя 1 неделю я утром проснулась с ужасной сыпью на лице, которая к вечеру обернулась еще и зудом. Я позвонила врачу и она мне сказала, да, бывает и такое. Попробуйте отменить их на неделю, посмотрим что будет. Я так и сделала(+пила средства от аллергии, и мазала прыщики). Через дней 5 почти все прошло кроме оставшиеся небольшой сыпи. Пить их больше не стала. После узнала у дерматолога, что эти витамины сплошная химия! Ничего хорошего в них нет! Ах да, меня еще по утрам от них тошнило! Не советую никому, никогда!  Купите л

  0%|▏                                          | 3/734 [00:01<04:53,  2.49it/s]

...

Порошок при простуде Stada "Гриппостад" - Не помогает
____________________
Порошок при простуде Stada "Гриппостад" принимала в начале как только замечала признаки простуды. В других более тяжелых случаях конечно это нэеффективное средство, так как сейчас многие вирусные заболевания имеют очень агрессивную форму. Часто якобы обычная простуда переходит в серьезные воспаления легких или тяжелый бронхит. Поэтому лечится Гриппостадом не рекомендую и экспер. И если вначале не помогает парошок, то следует обратится к врачу для более серьезного назначения лекарственных средств. Если же простуда легкая, не вирусная инфекция, то в один или два приема действие Гриппостада должно быть ощутима. Цена порошка не очень дорогая при том, что можно покупать по одному или двум пакетикам. Советую, но при очень легкой простуде. 


  1%|▍                                          | 7/734 [00:02<04:13,  2.86it/s]

Противовирусный препарат Ингавирин капсулы - деньги на ветер
____________________
Два дня назад появились продромальные явления, слабость, вялость, дискомфорт в носоглотке и пока совсем не слег заглянул в аптеку. В аптеке посоветовали Ингаверин - этот препарат, на сегодня самый назначаемый. Цена вопроса 477 руб. Но как говорится в известном анекдоте: - «ложки нашлись, а осадок остался» не подействовало. На 8-й день выпил наш отечественный растворимый антигриппин + арбидол и через 8 часов почувствовал себя гораздо лучше. Не тратьте деньги и время, есть проверенные и более действенные лекарственные препараты. Мое мнение о ингаверине ничего личного – только бизнес. Надеюсь мой отзыв о ингаверине будет Вам полезен. Год выпуска/покупки: 2014 


  1%|▍                                          | 7/734 [00:02<04:55,  2.46it/s]


KeyboardInterrupt: 

In [118]:
results = rouge.compute(
        predictions=summaries,
        references=data["test"]['summary']
    )

In [119]:
results

{'rouge1': 0.06561567406254055,
 'rouge2': 0.02861035422343324,
 'rougeL': 0.06514207862981704,
 'rougeLsum': 0.06472687167510056}

In [124]:
summaries[10]

'Противовирусный препарат Bittner Richard Афлубин - Хороший препарат'

In [125]:
data["test"]['summary'][10]

'Противовирусные таблетки для рассасывания Эргоферон - Действенный'

In [129]:
data["test"]

Dataset({
    features: ['summary', 'text'],
    num_rows: 734
})

In [35]:
test_review[0]

'С середины лета моя дочь в год и десять месяцев начала посещать детский сад, начался знакомый всем мамочкам, имеющим детей адаптационный период-привыкание не только к тому. что ребенок остается без любимой мамули, с которой он неразрывно существовал девять месяцев до рождения и соответственно после рождения..., а также к новым лицам-воспитателя, нянечкам и детишкам... Я надеялась что все обойдется без особых заболеваний. но сразу же после первой недели мы начали болеть... да болеть так что обычный гомеопатический препарат "анаферон" абсолютно не смог оказать на ее органзим никакого воздействия. напротив. может мне так только показалось. но после приема таблетки, начинала стремительно подниматься температура... В конце концов пришлось лечиться антибиотиками...( Время использования: 5 дней '

In [26]:
test_target[0]

'Таблетки гомеопатические Анаферон детский - противовирусный препарат...'

In [32]:
for i in range(len(test_review)):
    if test_review[i].find('Давно страдаю запорами, не знаю причину. Обычно не обращала на это особого внимания,')!=-1:
        print(i)

415


In [33]:
test_target[415]

'Слабительное Бисакодил-Хемофарм - Сильный, но...'